# A Demonstration of Typical and Topical Similarities in Neural Text Embedding Spaces

The notion of similarity between two pieces of text in a learned embedding space is a function of how the embedding model is trained. Is _yale_ closer to _harvard_, or to _alumni_? Among the many possible notions of relatedness type-based (_yale_ to _harvard_) and topic-based (_yale_ to _alumni_) similarities are more popularly studied in the literature. The [DESM paper](https://arxiv.org/abs/1602.01137) (2016) referred to these two relationships as _Typical_ and _Topical_ similarities, respectively, in the context of the popular [word2vec](https://arxiv.org/abs/1301.3781.pdf) model. However, the idea of _Typical_ and _Topical_ similarities goes at least as far back as [Saussure](https://en.wikipedia.org/wiki/Ferdinand_de_Saussure) who referred to them under the (dare we say) slightly harder to enunciate names of [_Syntagmatic_ and _Paradigmatic_](https://en.wikipedia.org/wiki/Course_in_General_Linguistics#Syntagmatic_and_paradigmatic_relations) relations. Other, recent works (e.g., [Sun et al.](http://www.aclweb.org/anthology/P15-1014)) have also distinguished between these two relationships in the context of word embedding spaces.

Obviously, the notion of _Typical_ and _Topical_ similarities goes beyond just words. This demonstration explores the same relationships in the context of short-text embeddings. Both the _Topical_ and the _Typical_ model architectures in this demo are based on the [CDSSM model](http://dl.acm.org/citation.cfm?id=2661935). However, while the _Topical_ model is trained on query-document pairs as specified by the original paper, the _Typical_ model is trained on query prefix-suffix pairs as proposed by [Mitra and Craswell](http://dl.acm.org/citation.cfm?id=2806599).

Last but not the least, we also demonstrate that the _Typical_-CDSSM model in particular is capable of performing analogies over short-text using simple vector algebra in line with what was reported by [Mitra](http://dl.acm.org/citation.cfm?id=2767702).

## Let's begin...

In [ ]:
from __future__ import print_function
import sys
import os
import csv
import re
import time
import math
import random
import operator
import pprint
import numpy as np
import cntk as C
from cntk import graph
from IPython.display import HTML, display

C.set_default_device(C.cpu())

## Data reader

In [ ]:
class Sample:
    
    def __init__(self):
        self.source = ""
        self.targets = []
    
class DataReader:
    max_words = 10
    
    def __init__(self, data_file, ngraphs_file, num_negs):
        self.__load_ngraphs(ngraphs_file)
        self.data_file = open(data_file, mode='r')
        self.num_negs = num_negs
    
    def __load_ngraphs(self, filename):
        self.ngraphs = {}
        self.max_ngraph_len = 0
        with open(filename, mode='r') as f:
            reader = csv.reader(f, delimiter='\t')
            for row in reader:
                self.ngraphs[row[0]] = int(row[1]) - 1
                self.max_ngraph_len = max(self.max_ngraph_len, len(row[0]))
        self.num_ngraphs = len(self.ngraphs)

    def __read_samples(self, num_samples):
        labels = np.zeros((num_samples, self.num_negs+1))
        samples_qd = []
        samples_ps = []
        mb_size = 0
        for i in range(num_samples):
            query = ""
            doc = ""
            query_words = []
            num_words = 0
            while num_words < 2:
                row = self.data_file.readline()
                if row == "":
                    self.data_file.seek(0)
                else:
                    row = re.sub('[^0-9a-z\t]+', ' ', row.lower())
                    cols = row.split('\t')
                    query = cols[0]
                    doc = cols[1]
                    query_words = query.split(' ')
                    num_words = len(query_words)
            if num_words < 2:
                break
            num_words_p = random.randint(1, num_words-1)
            prefix = ' '.join(query_words[:num_words_p])
            suffix = ' '.join(query_words[num_words_p:])
            
            curr_sample_qd = Sample()
            curr_sample_qd.source = query
            curr_sample_qd.targets.append(doc)
            samples_qd.append(curr_sample_qd)
            
            curr_sample_ps = Sample()
            curr_sample_ps.source = prefix
            curr_sample_ps.targets.append(suffix)
            samples_ps.append(curr_sample_ps)

            labels[i][0] = 1
            mb_size += 1

        for i in range(num_samples):
            for j in range(1, self.num_negs+1):
                samples_qd[i].targets.append(samples_qd[(i+j)%num_samples].targets[0])
                samples_ps[i].targets.append(samples_ps[(i+j)%num_samples].targets[0])
                
        return samples_qd, samples_ps, labels, mb_size
        
    def __get_ngraph_features(self, samples):
        features_src = np.zeros((len(samples), self.num_ngraphs, self.max_words))
        features_tgts = np.zeros((len(samples), self.num_negs+1, self.num_ngraphs, self.max_words))
        for sample_idx, sample in enumerate(samples):
            # loop over source and targets -- tgt_idx = 0 corresponds to source 
            for tgt_idx in range(len(sample.targets)+1):
                tgt = sample.source if tgt_idx == 0 else sample.targets[tgt_idx-1]
                for w_idx, word in enumerate(tgt.split()):
                    token = '#' + word + '#'
                    token_len = len(token)
                    for i in range(token_len):
                        for j in range(0, self.max_ngraph_len):
                            if i+j < token_len:
                                ngraph_idx = self.ngraphs.get(token[i:i+j])
                                if ngraph_idx != None:
                                    if tgt_idx == 0:
                                        features_src[sample_idx, ngraph_idx, min(w_idx, self.max_words-1)] += 1
                                    else:
                                        features_tgts[sample_idx, tgt_idx-1, ngraph_idx, min(w_idx, self.max_words-1)] += 1
        return features_src, features_tgts

    def get_minibatch(self, num_samples):
        samples_qd, samples_ps, labels, mb_size = self.__read_samples(num_samples)
        features_qd_src, features_qd_tgts = self.__get_ngraph_features(samples_qd)
        features_ps_src, features_ps_tgts = self.__get_ngraph_features(samples_ps)
        return features_qd_src, features_qd_tgts, features_ps_src, features_ps_tgts, labels, mb_size

    def get_test_minibatches(self, candidates_file, mb_size):        
        empty_targets = []
        for i in range(self.num_negs+1):
            empty_targets.append("")
        
        query = ""
        with open(candidates_file, mode='r') as f:
            while True:
                samples = []
                candidates = []
                curr_mb_size = 0
                while curr_mb_size < mb_size:
                    query = f.readline().strip()
                    if query == "":
                        break
                    curr_sample = Sample()
                    curr_sample.source = query
                    curr_sample.targets = empty_targets
                    samples.append(curr_sample)
                    candidates.append(query)
                    curr_mb_size += 1
                features_src = None
                features_tgts = None
                if curr_mb_size != 0:
                    features_src, features_tgts = self.__get_ngraph_features(samples)
                yield features_src, candidates, curr_mb_size
                if query == "":
                        break

## The CDSSM model

In [ ]:
def get_embedding_model(num_words, num_hidden_nodes, convolutional):
    if convolutional:
        word_window_size = 3
        pooling_kernel_width = num_words - word_window_size + 1 # = 8
        return C.Sequential ([
                C.Convolution((word_window_size, 1), num_hidden_nodes, activation=C.tanh, strides=(1, 1), pad=False),
                C.MaxPooling((pooling_kernel_width, 1), strides=(1, 1), pad=False),
                #C.Dense(num_hidden_nodes, activation=C.tanh),
                C.Dense(num_hidden_nodes, activation=C.tanh)])
    else:
        return C.Sequential ([
                C.Dense(num_hidden_nodes, activation=C.tanh),
                #C.Dense(num_hidden_nodes, activation=C.tanh),
                C.Dense(num_hidden_nodes, activation=C.tanh)])
    
def dssm(features_src, features_tgts, num_ngraphs, num_words, num_negs, num_hidden_nodes, convolutional):
    const_gamma = C.constant(10.0, shape=(1, 1))
    embed_src   = get_embedding_model(num_words, num_hidden_nodes, convolutional)
    embed_tgt   = get_embedding_model(num_words, num_hidden_nodes, convolutional)
    net_src     = C.reshape(features_src, (num_ngraphs, num_words, 1))
    net_src     = embed_src(net_src)
    net_src     = C.alias(net_src, name='source_embedding')
    net_tgts    = [C.slice(features_tgts, 0, idx, idx+1) for idx in range(0, num_negs+1)]
    net_tgts    = [C.reshape(net_tgt, (num_ngraphs, num_words, 1)) for net_tgt in net_tgts]
    net_tgts    = [embed_tgt(net_tgt) for net_tgt in net_tgts]
    net_tgts    = [C.cosine_distance(net_src, net_tgt) for net_tgt in net_tgts]
    net_tgts    = C.splice(net_tgts)
    net_tgts    = C.element_times(net_tgts, const_gamma)

    return net_tgts

## Train

In [ ]:
def train(train_file, ngraphs_file, num_negs, minibatch_size, minibatches_per_epoch, epochs, learning_rate, num_hidden_nodes, convolutional, test_query):
    
    # initialize train data readers
    reader = DataReader(train_file, ngraphs_file, num_negs)
       
    # input variables denoting the features and label data
    features_src  = C.input_variable((reader.num_ngraphs, reader.max_words), np.float32)
    features_tgts = C.input_variable((reader.num_negs+1, reader.num_ngraphs, reader.max_words), np.float32)
    labels        = C.input_variable((reader.num_negs+1), np.float32)

    # Instantiate the Topical model and specify loss function
    z_top   = dssm(features_src, features_tgts, reader.num_ngraphs, reader.max_words, reader.num_negs, num_hidden_nodes, convolutional)
    ce_top  = C.cross_entropy_with_softmax(z_top, labels)
    pe_top  = C.classification_error(z_top, labels)
    emb_top = C.combine([z_top.find_by_name("source_embedding").owner])

    # Instantiate the Typical model and specify loss function
    z_typ   = dssm(features_src, features_tgts, reader.num_ngraphs, reader.max_words, reader.num_negs, num_hidden_nodes, convolutional)
    ce_typ  = C.cross_entropy_with_softmax(z_typ, labels)
    pe_typ  = C.classification_error(z_typ, labels)
    emb_typ = C.combine([z_typ.find_by_name("source_embedding").owner])

    # Instantiate the trainer objects to drive the model training
    lr_per_minibatch = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
    trainer_top = C.Trainer(z_top, ce_top, pe_top, [C.sgd(z_top.parameters, lr=lr_per_minibatch)])
    trainer_typ = C.Trainer(z_typ, ce_typ, pe_typ, [C.sgd(z_typ.parameters, lr=lr_per_minibatch)])

    pp_top = C.ProgressPrinter(freq=10, tag='Training', gen_heartbeat=False, num_epochs=epochs)
    pp_typ = C.ProgressPrinter(freq=10, tag='Training', gen_heartbeat=False, num_epochs=epochs)

    for i in range(epochs+1):
        
        print('\nafter {} epoch(s)'.format(i))
        display_neighbours(emb_top, emb_typ, reader, minibatch_size)
        
        if i < epochs:
            # train epoch
            for j in range(minibatches_per_epoch):
                train_features_query, train_features_docs, train_features_prefix, train_features_suffixes, train_labels, actual_mb_size = reader.get_minibatch(minibatch_size)
                trainer_top.train_minibatch({features_src : train_features_query, features_tgts : train_features_docs, labels : train_labels})
                trainer_typ.train_minibatch({features_src : train_features_prefix, features_tgts : train_features_suffixes, labels : train_labels})
                pp_top.update_with_trainer(trainer_top, with_metric=True)
                pp_typ.update_with_trainer(trainer_typ, with_metric=True)

            pp_top.epoch_summary(with_metric=True)
            pp_typ.epoch_summary(with_metric=True)

    return z_top, z_typ

def display_neighbours(emb_top, emb_typ, reader, minibatch_size):
    embeddings_top = {}
    scores_top = {}
    embeddings_typ = {}
    scores_typ = {}
    test_minibatches = reader.get_test_minibatches("data\\queries-wiki.txt", minibatch_size)
    while True:
        test_features, candidates, actual_mb_size = next(test_minibatches)
        if actual_mb_size > 0:
            result_top = emb_top.eval({emb_top.arguments[0] : test_features})
            result_top = result_top.reshape((actual_mb_size, num_hidden_nodes))
            result_typ = emb_typ.eval({emb_typ.arguments[0] : test_features})
            result_typ = result_typ.reshape((actual_mb_size, num_hidden_nodes))
            for k in range(len(candidates)):
                embeddings_top[candidates[k]] = result_top[k]
                embeddings_typ[candidates[k]] = result_typ[k]
        if actual_mb_size < minibatch_size:
            break
    test_embedding_top = embeddings_top[test_query]
    test_embedding_typ = embeddings_typ[test_query]
    for k,v in embeddings_top.items():
        if k != test_query:
            scores_top[k] = cosine_sim(test_embedding_top, v)
    for k,v in embeddings_typ.items():
        #if k != test_query:
        scores_typ[k] = cosine_sim(test_embedding_typ, v)
    sorted_candidates_top = sorted(scores_top.items(), key=operator.itemgetter(1), reverse=True)
    sorted_candidates_typ = sorted(scores_typ.items(), key=operator.itemgetter(1), reverse=True)
    display(HTML('<table><tr><th>Topical neighbours</th><th>Typical neighbours</th></tr><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td><td>{}</td>'.format(sorted_candidates_top[k][0], sorted_candidates_typ[k][0])for k in range(10)))))
    
def cosine_sim(a, b):
    d = dot_product(a, b);
    na = max(1e-20, norm(a));
    nb = max(1e-20, norm(b));
    return (d / (na * nb) + 1) / 2;

def dot_product(a, b):
    result = 0;
    for i in range(len(a)):
        result += (a[i] * b[i])
    return result;
    
def norm(a):
    return math.sqrt(dot_product(a, a))

In [ ]:
minibatch_size = 1024
minibatches_per_epoch = 128
epochs = 8
train_file = "data\\query-titles.txt"
ngraphs_file = "data\\ngraphs.txt"
num_negs = 4
learning_rate = 0.5
num_hidden_nodes = 128
convolutional = False
test_query = "seattle"

model_top, model_typ = train(train_file, ngraphs_file, num_negs, minibatch_size, minibatches_per_epoch, epochs, learning_rate, num_hidden_nodes, convolutional, test_query)
print('bye!')